In [ ]:
from labdata.schema import *
from labdata import chronic_paper


In [ ]:
# TODO: add missing procedures for mice
insertions_query = Procedure() & 'procedure_type = "chronic implant"'
chronic_mice = insertions_query.fetch('subject_name')
chronic_sessions_query = EphysRecording() & [dict(subject_name = c) for c in chronic_mice]
n_sorted_prbs = len(SpikeSorting() & 'parameter_set_num = 5')

chronic_probe_ids = np.unique((EphysRecording.ProbeSetting() & chronic_sessions_query).fetch('probe_id')).astype(int)

insertions = ((pd.DataFrame((EphysRecording.ProbeSetting() & chronic_sessions_query).proj('probe_id').fetch())[
               ['subject_name','probe_id']].drop_duplicates()))


print(f'We performed {len(insertions)} chronic insertions ({len(chronic_probe_ids)} probes) into {len(chronic_mice)} mice and recorded {len(chronic_sessions_query)} sessions.')
print(f'We spike sorted  {n_sorted_prbs} probe*sessions.')
# insertions

In [ ]:
len(((ProbeExtraction() & 'extraction_successful = 1')))/len(ProbeExtraction())